## BTCNet

Welcome to BTCNet, a Bitcoin trading bot using machine learning.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

We create a long short term memory recurrent neural network to predict the price of Bitcoin 1 hour in the future. We train the network on historical hourly pricing data along with volume.

Thesis: The price of Bitcoin can be sufficiently predicted by momentum and volume.

FUD, FOMO, Whales, and manipulation lead to large price swings driven by momentum in short periods of time.

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Import the historical price data.

We create our own Dataset class for our dataset.

In [49]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class BTCDataset(Dataset):
    def __init__(self, csv):
        self.btcFrame = pd.read_csv(csv)
        self.targets = self.btcFrame.iloc[:, 4:5]
        
    def __len__(self):
        return len(self.btcFrame.iloc[0])
    
    def __getitem__(self, i):
        item = self.btcFrame.iloc[i, 1:2].values
        #item = item.astype("float")
        return item
    
    def getTarget(self, i):
        target = self.targets.iloc[i, :].values
        target = target.astype("float")
        return target

Now we import our dataset from a csv file.

In [50]:
btcData = BTCDataset("bitcoinPrices.csv")

for i in range(10):
    sample = btcData[i]
    print(sample, f"Target is {btcData.getTarget(i)}")

[300.] Target is [300.]
[300.] Target is [300.]
[300.] Target is [300.]
[300.] Target is [300.]
[300.] Target is [300.]
[300.] Target is [300.]
[300.] Target is [300.]
[300.] Target is [300.]
[300.] Target is [300.]
[300.] Target is [300.]


In [51]:
batch_size = 10

dataloader = DataLoader(btcData, batch_size=batch_size, shuffle=False, num_workers=2)

for i, sample in enumerate(dataloader):
    print(i, sample)
    
    if i == 10:
        break

0 tensor([[300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.]], dtype=torch.float64)


In [48]:
class BTCBot(nn.Module):
    def __init__(self, input_size, batch_size, hidden_size, output_size):
        super(BTCBot, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
        self.fc1 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
        self.hidden = (torch.zeros(1, batch_size, hidden_size), torch.zeros(1, batch_size, hidden_size))
        
    def forward(self, input):
        lstm_out, self.hidden = self.lstm(input, self.hidden)
        lin_out = self.fc1(lstm_out)
        out = self.relu(lin_out)
        return out

In [53]:
btcModel = BTCBot(input_size=len(dataloader), batch_size=batch_size, hidden_size=5, output_size=1)
for i, sample in enumerate(dataloader):
    out = btcModel(sample.view(1, batch_size, -1))
    print(out)

RuntimeError: invalid argument 2: size '[1 x 10 x -1]' is invalid for input with 8 elements at /Users/soumith/miniconda2/conda-bld/pytorch_1532623076075/work/aten/src/TH/THStorage.cpp:80